# Vertex Document AI — PDF’den Key-Value Çıkarma (Notebook)

Bu notebook, bir PDF (örn. fatura) dosyasını **Vertex Document AI** ile işleyip, dönen sonuçtan **key-value** alanlarını çıkarır ve **JSON** olarak kaydeder.

> Not: Notebook “hızlı iterasyon” için tasarlandı. Üretim tarafına geçerken aynı mantığı servis (FastAPI/Cloud Run) içine taşıyabilirsin.


## 0) Ön Koşullar

- Google Cloud projesi (Project)
- Document AI API etkin
- Bir **Processor** (General Form Extractor veya Invoice Processor vb. Burada Form Parser kullanılabilir)
- Yetkilendirme:
  - **Yerel ortamda:** `gcloud auth application-default login`
  - **Service Account ile:** JSON key dosyasını indirip `GOOGLE_APPLICATION_CREDENTIALS` ile işaretle


In [53]:
# Eğer ortamında kurulu değilse:
%pip install google-cloud-documentai

# Kurulumdan sonra kernel yeniden başlatman gerekebilir.



[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [55]:
import os

# Seçenek A) ADC (Application Default Credentials) — önerilen hızlı yol:
# Terminalde bir kez çalıştır:
#   gcloud auth application-default login
#
# Seçenek B) Service Account JSON key dosyası ile:
# /Users/cemguler/.config/gcloud/application_default_credentials.json
# 
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/cemguler/.config/gcloud/application_default_credentials.json"
#print(os.environ.get("GOOGLE_APPLICATION_CREDENTIALS"))
print("Auth hazır. (ADC veya SERVICE ACCOUNT seçeneğinden biri aktif olmalı)")


Auth hazır. (ADC veya SERVICE ACCOUNT seçeneğinden biri aktif olmalı)


## 1) Konfigürasyon

Aşağıdaki değerleri kendi projenize göre doldurun:

- `project_id`: GCP Proje ID
- `location`: Processor lokasyonu (örn. `us` veya `eu`)
- `processor_id`: Document AI Processor ID
- `file_path`: İşlenecek PDF dosya yolu


In [56]:
# --- CONFIG ---
project_id   = "vertextraining-486212"
location     = "us"            # region önemli
processor_id = "a5a288a99fdb0317"

file_path    = "./data/ornek_fatura.pdf"    

# Çıktı JSON dosyası
output_json_path = "./data/documentai_kv_output.json"


## 2) Dokümanı İşle (Process Document)

Document AI, PDF’i parse eder ve:
- Form fields (key-value)
- Entities (özellikle invoice gibi processor’larda)
- Metin + layout bilgilerini döner.

Bu bölümde API çağrısını yapıyoruz.


In [58]:
from google.cloud import documentai_v1 as documentai

# Client
client = documentai.DocumentProcessorServiceClient(
    client_options={"api_endpoint": f"{location}-documentai.googleapis.com"}
)

# Processor resource name
name = client.processor_path(project_id, location, processor_id)

# Dosyayı oku
with open(file_path, "rb") as f:
    file_content = f.read()

# PDF için mime_type: application/pdf
raw_document = documentai.RawDocument(content=file_content, mime_type="application/pdf")

request = documentai.ProcessRequest(name=name, raw_document=raw_document)

result = client.process_document(request=request)
document = result.document

print("✅ İşlem tamamlandı.")
print("Text length:", len(document.text or ""))
print("Text: ", document.text[:250])  # İlk 250 karakteri göster

# For a full list of `Document` object attributes, reference this page:
# https://cloud.google.com/document-ai/docs/reference/rest/v1/Document


✅ İşlem tamamlandı.
Text length: 698
Text:  ÖRNEK FATURA
Document Al form key/val çıkarımı için etiketli örnek pdf.
Satıcı
Alıcı
ByteFlow Bilişim LTD
Vergi No: 123456789
Adres: Ataşehir, İstanbul, TR
E-posta: billing@byteflow.tr
ACME Corp
Vergi No: 987654321
Adres: Levent, İstanbul, TR
E-posta


## 3) Key-Value Alanlarını Çıkar

Document AI sonuçlarında **form fields** genellikle:
- `document.pages[].form_fields[]` altında yer alır.
- Her form field: `field_name` (key) ve `field_value` (value) içerir.

Aşağıdaki yardımcı fonksiyonlar:
- Layout segmentlerinden metni çeker
- Tüm sayfalardaki key-value’ları tek listeye indirger


In [59]:
from typing import List, Dict, Any, Optional

def layout_to_text(layout: Any, full_text: str) -> str:
    """Document AI layout.text_anchor üzerinden metni çıkar."""
    if not layout or not getattr(layout, "text_anchor", None):
        return ""
    segments = getattr(layout.text_anchor, "text_segments", None) or []
    out = []
    for seg in segments:
        start = int(seg.start_index) if seg.start_index is not None else 0
        end   = int(seg.end_index) if seg.end_index is not None else 0
        out.append(full_text[start:end])
    return "".join(out).strip()

def extract_form_fields(document: Any) -> List[Dict[str, Any]]:
    full_text = document.text or ""
    kvs = []
    for page_idx, page in enumerate(document.pages or []):
        for ff in page.form_fields or []:
            key_text = layout_to_text(ff.field_name, full_text)
            val_text = layout_to_text(ff.field_value, full_text)
            confidence = float(getattr(ff, "confidence", 0.0) or 0.0)

            if key_text or val_text:
                kvs.append({
                    "page": page_idx + 1,
                    "key": key_text,
                    "value": val_text,
                    "confidence": confidence
                })
    return kvs

kvs = extract_form_fields(document)
print(f"✅ Bulunan key-value sayısı: {len(kvs)}")

# İlk 10 kaydı göster
for item in kvs:
    print(item)


✅ Bulunan key-value sayısı: 17
{'page': 1, 'key': 'E-posta:', 'value': 'finance@acme.corp', 'confidence': 0.0}
{'page': 1, 'key': 'Adres:', 'value': 'Levent, İstanbul, TR', 'confidence': 0.0}
{'page': 1, 'key': 'E-posta:', 'value': 'billing@byteflow.tr', 'confidence': 0.0}
{'page': 1, 'key': 'Adres:', 'value': 'Ataşehir, İstanbul, TR', 'confidence': 0.0}
{'page': 1, 'key': 'Vergi No:', 'value': '123456789', 'confidence': 0.0}
{'page': 1, 'key': 'Vergi No:', 'value': '987654321', 'confidence': 0.0}
{'page': 1, 'key': 'Fatura Tarihi', 'value': '02.02.2026', 'confidence': 0.0}
{'page': 1, 'key': 'Para Birimi', 'value': 'TRY', 'confidence': 0.0}
{'page': 1, 'key': 'Vade Tarihi', 'value': '16.02.2026', 'confidence': 0.0}
{'page': 1, 'key': 'IBAN:', 'value': 'TR00 0000 0000 0000 0000 0000 00', 'confidence': 0.0}
{'page': 1, 'key': 'Genel Toplam', 'value': '4.200,00', 'confidence': 0.0}
{'page': 1, 'key': 'KDV Tutarı', 'value': '700,00', 'confidence': 0.0}
{'page': 1, 'key': 'KDV Oranı', 'val

## 4) (Opsiyonel) Entities Çıkarımı (Invoice Processor vb.)

Bazı specialized processor’lar, form field yerine daha “anlamlı” alanları `entities` altında verir.
Örn: invoice_date, supplier_name, total_amount gibi.


In [60]:
def extract_entities(document: Any) -> List[Dict[str, Any]]:
    ents = []
    for e in document.entities or []:
        ents.append({
            "type": getattr(e, "type_", None) or getattr(e, "type", None),
            "mention_text": getattr(e, "mention_text", ""),
            "normalized_value": getattr(getattr(e, "normalized_value", None), "text", None),
            "confidence": float(getattr(e, "confidence", 0.0) or 0.0)
        })
    return ents

entities = extract_entities(document)
print(f"✅ Bulunan entity sayısı: {len(entities)}")

# İlk 15 entity
for e in entities[:15]:
    print(e)


✅ Bulunan entity sayısı: 1
{'type': 'generic_entities', 'mention_text': '', 'normalized_value': '', 'confidence': 0.0}


## 5) Basit Normalize (Fatura Örneği)

Hedef: JSON şema üretmek.
- Form fields’den veya entities’den bilgi çekebilirsin.
- Burada basit bir yaklaşım: entities varsa önce entities’yi kullan, yoksa key-value’da arama yap.

> Gerçek hayatta normalize için regex + sözlük + eşleştirme kuralları kullanılır.


In [61]:
import re
from collections import defaultdict

def pick_entity(entities: List[Dict[str, Any]], wanted_types: List[str]) -> Optional[Dict[str, Any]]:
    for wt in wanted_types:
        for e in entities:
            if (e.get("type") or "").lower() == wt.lower():
                return e
    return None

def find_in_kvs(kvs: List[Dict[str, Any]], key_patterns: List[str]) -> Optional[Dict[str, Any]]:
    for pat in key_patterns:
        rx = re.compile(pat, re.IGNORECASE)
        for kv in kvs:
            if rx.search(kv.get("key","")):
                return kv
    return None

# 1) invoice_date
invoice_date = None
kv = find_in_kvs(kvs, [r"tarih", r"invoice\s*date", r"date"])
invoice_date = kv["value"] if kv else None

# 2) total_amount
total_amount = None
kv = find_in_kvs(kvs, [r"toplam", r"genel\s*toplam", r"total\s*amount", r"amount\s*due"])
total_amount = kv["value"] if kv else None

normalized = {
    "invoice_date": invoice_date,
    "total_amount": total_amount,
    "source": {
        "used_entities": len(entities) > 0,
        "processor": {"project_id": project_id, "location": location, "processor_id": processor_id},
        "file_path": file_path
    }
}

normalized


{'invoice_date': '02.02.2026',
 'total_amount': '4.200,00',
 'source': {'used_entities': True,
  'processor': {'project_id': 'vertextraining-486212',
   'location': 'us',
   'processor_id': 'a5a288a99fdb0317'},
  'file_path': './data/ornek_fatura.pdf'}}

## 6) JSON Export

- Ham key-value listesi
- Ham entities listesi
- Normalize edilmiş alanlar

Tek dosyada kaydediyoruz.


In [ ]:
import json

payload = {
    "normalized": normalized,
    "key_values": kvs
}

with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(payload, f, ensure_ascii=False, indent=2)

print(f"✅ JSON kaydedildi: {output_json_path}")
